# Supply Chain & Inventory Forecasting with AI Advisor

Muh Amri Sidiq

This notebook demonstrates a complete workflow for demand forecasting and AI-driven order recommendations.
The system combines:
- Data ingestion & preprocessing
- Three forecasting models (Prophet, XGBoost, Hybrid)
- Business-driven inputs (promo boost, storage cost, selling price)
- AI-powered advisor (RAG) for supplier & contract analysis
- Automated PDF report generation (with charts)

The final version is deployed as a Streamlit app for interactive use, but this notebook shows the full pipeline step by step.

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error
from fpdf import FPDF
from io import BytesIO

# For AI Advisor
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

## Load & Explore Data

We'll load historical sales data and inspect the structure.

In [ ]:
# Example CSV structure: date, family, sales
df = pd.read_csv('sales_data.csv', parse_dates=['date'])
df.head()

## Forecasting Models

We'll use three approaches:
- Prophet (trend + seasonality)
- XGBoost (time-series features)
- Hybrid (Prophet for trend + XGBoost for residuals)

Each model will be evaluated using MAPE, and the best model will be selected automatically.

In [ ]:
# Prophet Forecast Function
def run_prophet(df_item, forecast_period, use_holidays=True):
    df_train = df_item.tail(int(forecast_period * 1.5))
    model = Prophet(yearly_seasonality=True, weekly_seasonality=True, holidays_prior_scale=10) if use_holidays else Prophet()
    if use_holidays:
        model.add_country_holidays(country_name='ID')
    model.fit(df_train)
    future = model.make_future_dataframe(periods=forecast_period)
    forecast = model.predict(future)
    merged = forecast.set_index('ds')[['yhat']].join(df_item.set_index('ds')[['y']], how='left')
    merged['y'] = merged['y'].replace(0, np.nan).fillna(method='ffill')
    hist = merged.dropna()
    mape = mean_absolute_percentage_error(hist['y'], hist['yhat']) * 100
    return forecast.set_index('ds')[['yhat']], mape

# XGBoost Forecast Function
def run_xgb(df_item, forecast_period):
    train_window = int(forecast_period * 1.5)
    df_train = df_item.tail(train_window).copy()
    df_train['day'] = df_train['ds'].dt.day
    df_train['month'] = df_train['ds'].dt.month
    df_train['dow'] = df_train['ds'].dt.dayofweek
    df_train['lag_7'] = df_train['y'].shift(7)
    df_train['promo'] = 0
    df_train = df_train.dropna()
    X = df_train[['day', 'month', 'dow', 'lag_7', 'promo']]
    y = df_train['y']
    split = int(len(df_train) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    model = xgb.XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.05)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    last_date = df_item['ds'].max()
    forecast_dates = [last_date + pd.Timedelta(days=i+1) for i in range(forecast_period)]
    preds, current_lag = [], df_train.iloc[-1]['lag_7']
    for date in forecast_dates:
        feat = [date.day, date.month, date.dayofweek, current_lag, 0]
        pred = model.predict(pd.DataFrame([feat], columns=['day', 'month', 'dow', 'lag_7', 'promo']))[0]
        preds.append(pred)
        current_lag = pred
    forecast = pd.DataFrame({'ds': forecast_dates, 'yhat': preds}).set_index('ds')
    return forecast, mape

## Business Inputs

Simulate promo impact, storage cost, and revenue gain to integrate business context into the forecast.

In [ ]:
storage_cost_per_unit = 500
selling_price_per_unit = 10000
promo_boost_pct = 20

avg_sales = df['sales'].mean()
extra_sales = avg_sales * (promo_boost_pct / 100)
storage_cost = storage_cost_per_unit * extra_sales
revenue_gain = selling_price_per_unit * extra_sales

print(f"Extra Sales: {extra_sales:.0f} units")
print(f"Extra Storage Cost: Rp {storage_cost:,.0f}")
print(f"Potential Revenue Gain: Rp {revenue_gain:,.0f}")

## AI Advisor (RAG)

Uses PDF supplier contracts and warehouse policies to generate order recommendations.

In [ ]:
pdf_files = ['supplier_policy.pdf']  # Example
api_key = 'YOUR_OPENAI_API_KEY'
texts = []
for pdf in pdf_files:
    reader = PdfReader(pdf)
    for page in reader.pages:
        texts.append(page.extract_text())
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.create_documents(texts)
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()
llm = OpenAI(temperature=0, openai_api_key=api_key)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

prompt = "Based on the forecast and policies, calculate optimal order quantity."
recommendation = qa_chain.run(prompt)
print("AI Recommendation:", recommendation)

## Export PDF Report

Generates a PDF summary including forecast metrics, business impact, and AI recommendations.

In [ ]:
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=14)
pdf.cell(200, 10, txt="Supply Chain & Inventory Report", ln=True, align='C')
pdf.ln(10)
pdf.multi_cell(0, 10, txt=f"Extra Sales: {extra_sales:.0f} units")
pdf.multi_cell(0, 10, txt=f"Extra Storage Cost: Rp {storage_cost:,.0f}")
pdf.multi_cell(0, 10, txt=f"Potential Revenue Gain: Rp {revenue_gain:,.0f}")
pdf.multi_cell(0, 10, txt=f"AI Recommendation: {recommendation}")
pdf.output("supply_chain_report.pdf")

## Conclussion

This project showcases a complete pipeline for demand forecasting and decision support in supply chain management.
By combining three forecasting models (Prophet, XGBoost, and Hybrid), the system automatically selects the most accurate model based on MAPE.

On top of forecasting, the tool integrates business impact analysis (storage cost, sales boost, revenue gain) and an AI-powered advisor that reads supplier contracts and warehouse policies to recommend optimal order quantities.

The result is a single, automated workflow that supports data-driven decision-making for inventory and procurement teams.
Future improvements could include real-time sales ingestion, multi-location optimization, and cloud deployment for scalability.